In [26]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1. CARGA
try:
    df = pd.read_parquet("data_cleaned/master_with_clusters.parquet")
except:
    df = pd.read_parquet("../data_cleaned/master_with_clusters.parquet")

df['Fecha'] = pd.to_datetime(df['Fecha'])

# 2. DEFINIR FECHA DE CORTE (SNAPSHOT)
# Usaremos el último mes de datos como "Futuro" para validar
fecha_fin_datos = df['Fecha'].max()
fecha_corte = fecha_fin_datos - pd.Timedelta(days=30) 

print(f"Fecha de Corte: {fecha_corte.date()}")
print(f"Periodo de Entrenamiento (Historia): Hasta {fecha_corte.date()}")
print(f"Periodo de Validación (Futuro): {fecha_corte.date()} al {fecha_fin_datos.date()}")

# 3. CONSTRUCCIÓN DE TARGET (¿Compró después del corte?)
# Filtramos ventas futuras
futuro = df[df['Fecha'] > fecha_corte]
clientes_que_compraron = futuro['ID_Cliente'].unique()

# 4. CONSTRUCCIÓN DE FEATURES (Solo con historia PASADA)
historia = df[df['Fecha'] <= fecha_corte].copy()

# A) RFM Clásico (Recencia, Frecuencia, Monto)
features = historia.groupby('ID_Cliente').agg({
    'Fecha': lambda x: (fecha_corte - x.max()).days, # Recencia (Días sin venir)
    'ID_Venta': 'nunique',                           # Frecuencia Total
    'ingreso': 'sum',                                # Monto Total (LTV)
    'Cluster_Cliente': 'first',                      # Perfil (Vital)
    'Región': 'first'                                # Contexto
}).reset_index()

features.columns = ['ID_Cliente', 'Recencia_Dias', 'Frecuencia_Total', 'Monto_Total', 'Cluster', 'Region']

# B) Tendencia Reciente (Últimos 60 días vs Vida)
# ¿Su frecuencia reciente es menor a su habitual?
historia_reiciente = historia[historia['Fecha'] > (fecha_corte - pd.Timedelta(days=60))]
frecuencia_reciente = historia_reiciente.groupby('ID_Cliente')['ID_Venta'].nunique().reset_index()
frecuencia_reciente.columns = ['ID_Cliente', 'Frecuencia_60d']

features = features.merge(frecuencia_reciente, on='ID_Cliente', how='left').fillna(0)

# C) Feature de "Velocidad de Fuga"
# Si Recencia > Promedio de días entre compras, riesgo alto
# Calculamos días promedio entre compras por cliente
def avg_days_between(x):
    if len(x) < 2: return 30 # Default si solo tiene 1 compra
    return (x.max() - x.min()).days / (len(x) - 1)

ciclo_compra = historia.groupby('ID_Cliente')['Fecha'].apply(avg_days_between).reset_index()
ciclo_compra.columns = ['ID_Cliente', 'Ciclo_Promedio_Dias']

features = features.merge(ciclo_compra, on='ID_Cliente', how='left')

# Factor de Retraso: ¿Qué tan tarde va respecto a su costumbre?
# Ejemplo: Si viene cada 7 días y lleva 14 sin venir, Factor = 2.0
features['Factor_Retraso'] = features['Recencia_Dias'] / (features['Ciclo_Promedio_Dias'] + 1)

# 5. UNIÓN FINAL Y TARGET
# Si el cliente está en la lista de "clientes_que_compraron", Churn = 0. Si no, Churn = 1.
features['Target_Churn'] = features['ID_Cliente'].isin(clientes_que_compraron).apply(lambda x: 0 if x else 1)

# ONE-HOT ENCODING (Asegurando que Clusters y Región entren al modelo)
df_final = pd.get_dummies(features, columns=['Cluster', 'Region'], prefix=['Cluster', 'Reg'])

print(f"Dataset Final: {df_final.shape}")
print(f"Tasa de Fuga Real: {df_final['Target_Churn'].mean():.1%}")
print("Columnas:", df_final.columns.tolist())

Fecha de Corte: 2024-11-30
Periodo de Entrenamiento (Historia): Hasta 2024-11-30
Periodo de Validación (Futuro): 2024-11-30 al 2024-12-30
Dataset Final: (326, 17)
Tasa de Fuga Real: 42.0%
Columnas: ['ID_Cliente', 'Recencia_Dias', 'Frecuencia_Total', 'Monto_Total', 'Frecuencia_60d', 'Ciclo_Promedio_Dias', 'Factor_Retraso', 'Target_Churn', 'Cluster_0', 'Cluster_1', 'Cluster_2', 'Reg_Buenos Aires', 'Reg_Centro', 'Reg_Cuyo', 'Reg_NEA', 'Reg_NOA', 'Reg_Patagonia']


In [27]:
import optuna
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import recall_score, roc_auc_score, precision_score

# 1. PREPARACIÓN DE DATOS (Usando tu df_final del paso anterior)
X = df_final.drop(['ID_Cliente', 'Target_Churn'], axis=1)
y = df_final['Target_Churn']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Calcular ratio para balancear (Positivos / Negativos)
# Esto le dice al modelo: "Equivocarse en un fugado cuesta X veces más"
scale_pos_weight = float(np.sum(y == 0)) / np.sum(y == 1)

print(f"Entrenando con {X_train.shape[0]} clientes. Peso de balanceo: {scale_pos_weight:.2f}")

# 2. OPTIMIZACIÓN CON OPTUNA
def objective_churn(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'auc',
        'scale_pos_weight': scale_pos_weight, # CRÍTICO para mejorar Recall
        'n_estimators': trial.suggest_int('n_estimators', 200, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'max_depth': trial.suggest_int('max_depth', 3, 8),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'n_jobs': -1,
        'random_state': 42,
        'verbosity': 0
    }
    
    # Validación Cruzada (Más robusto que un solo split)
    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = []
    
    for train_idx, val_idx in cv.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
        
        model = xgb.XGBClassifier(**params)
        model.fit(X_tr, y_tr)
        
        preds = model.predict_proba(X_val)[:, 1]
        scores.append(roc_auc_score(y_val, preds))
    
    return np.mean(scores)

print("Buscando el mejor modelo de Fuga...")
study = optuna.create_study(direction='maximize')
study.optimize(objective_churn, n_trials=50)

print("\nMEJORES PARÁMETROS:")
print(study.best_params)

Entrenando con 244 clientes. Peso de balanceo: 1.38

[I 2025-11-30 20:55:59,354] A new study created in memory with name: no-name-87733ece-00f6-4c42-8c9b-ba9535d38d5b



Buscando el mejor modelo de Fuga...


[I 2025-11-30 20:56:00,033] Trial 0 finished with value: 0.8561058465939567 and parameters: {'n_estimators': 379, 'learning_rate': 0.014687476502274848, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.7702038489848673, 'colsample_bytree': 0.8812789671227763, 'gamma': 3.9447280968098033}. Best is trial 0 with value: 0.8561058465939567.
[I 2025-11-30 20:56:00,503] Trial 1 finished with value: 0.7815126050420167 and parameters: {'n_estimators': 360, 'learning_rate': 0.022380155730290903, 'max_depth': 3, 'min_child_weight': 10, 'subsample': 0.708940663897388, 'colsample_bytree': 0.9542717989494474, 'gamma': 3.783543955970405}. Best is trial 0 with value: 0.8561058465939567.
[I 2025-11-30 20:56:00,970] Trial 2 finished with value: 0.8484325645151678 and parameters: {'n_estimators': 280, 'learning_rate': 0.1161529052558079, 'max_depth': 5, 'min_child_weight': 9, 'subsample': 0.7833384571398931, 'colsample_bytree': 0.9789274269356553, 'gamma': 0.4858447891542622}. Best is trial 0 with v


MEJORES PARÁMETROS:
{'n_estimators': 597, 'learning_rate': 0.010417156254686255, 'max_depth': 7, 'min_child_weight': 2, 'subsample': 0.6386158530886273, 'colsample_bytree': 0.6010085275487569, 'gamma': 2.302982165919052}


In [30]:
# ==============================================================================
# 5. GENERACIÓN DE LA "LISTA DE RESCATE" Y DASHBOARD FINAL
# ==============================================================================

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
# Usamos los parámetros ganadores
final_params = study.best_params
final_params['scale_pos_weight'] = scale_pos_weight
final_params['objective'] = 'binary:logistic'

print("Entrenando modelo definitivo...")
model_final = xgb.XGBClassifier(**final_params)
model_final.fit(X, y)

# 2. PREDICCIÓN (CLIENTES ACTIVOS HOY)
# Usamos 'latest_features' que derivamos del último estado conocido
latest_features = df_final.groupby('ID_Cliente').last().reset_index()

# Preparamos X para predecir (quitando IDs y Target)
X_pred = latest_features.drop(['ID_Cliente', 'Target_Churn'], axis=1)
ids = latest_features['ID_Cliente']

# Probabilidades Futuras
probs_future = model_final.predict_proba(X_pred)[:, 1]

# Aplicamos el Umbral de Negocio (0.45)
umbral_negocio = 0.45
preds_future = (probs_future >= umbral_negocio).astype(int)

# --- CORRECCIÓN AQUÍ: USAR NOMBRES DE COLUMNAS EXISTENTES ---
# Recuperamos el Cluster original de las dummies
cluster_cols = [c for c in latest_features.columns if c.startswith('Cluster_')]
# Si por alguna razón no hay dummies activas, ponemos 'Desconocido'
if len(cluster_cols) > 0:
    cluster_labels = latest_features[cluster_cols].idxmax(axis=1).str.replace('Cluster_', '')
else:
    cluster_labels = 'Global'

df_risk = pd.DataFrame({
    'ID_Cliente': ids,
    'Probabilidad_Fuga': probs_future,
    'Es_Riesgo': preds_future,
    'Cluster': cluster_labels,
    'Monto_Total_Historico': latest_features['Monto_Total'], # <--- CAMBIO: Usamos Monto_Total
    'Dias_Sin_Venir': latest_features['Recencia_Dias']
})

# Filtramos la "Lista de Oro"
lista_rescate = df_risk[df_risk['Es_Riesgo'] == 1].sort_values('Probabilidad_Fuga', ascending=False)

print(f"¡Alerta! Se detectaron {len(lista_rescate)} clientes en riesgo de fuga.")
print("Top 5 Clientes en Riesgo (Prioridad):")
print(lista_rescate.head())
# 3. DASHBOARD EJECUTIVO DE RETENCIÓN
# ------------------------------------------------------
# Configuración de subplots con tipos específicos
# Configuración de subplots
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{"type": "domain"}, {"type": "xy"}], [{"type": "xy"}, {"type": "xy"}]],
    subplot_titles=("1. Proporción de Cartera en Riesgo", "2. Distribución de Probabilidad de Fuga", 
                    "3. Top Factores de Fuga", "4. Riesgo por Segmento")
)

# A) Donut Chart
riesgo_counts = df_risk['Es_Riesgo'].value_counts()
fig.add_trace(go.Pie(labels=['Seguros', 'En Riesgo'], values=[riesgo_counts.get(0,0), riesgo_counts.get(1,0)], 
                     hole=.6, marker_colors=['#27ae60', '#c0392b']), row=1, col=1)

# B) Histograma
fig.add_trace(go.Histogram(x=df_risk['Probabilidad_Fuga'], nbinsx=20, marker_color='#34495e', name='Clientes'), row=1, col=2)

# --- CORRECCIÓN MANUAL: Usamos add_shape en lugar de add_vline ---
fig.add_shape(
    type="line",
    x0=umbral_negocio, x1=umbral_negocio,
    y0=0, y1=1,
    yref='paper', # Ocupa toda la altura del subplot
    line=dict(color="red", width=2, dash="dash"),
    row=1, col=2
)
fig.add_annotation(
    x=umbral_negocio, y=1,
    yref='paper',
    text="Corte Acción",
    showarrow=False,
    font=dict(color="red"),
    row=1, col=2
)

# C) Feature Importance
importance = pd.DataFrame({
    'Feature': X.columns,
    'Gain': model_final.feature_importances_
}).sort_values('Gain', ascending=True).tail(10)

fig.add_trace(go.Bar(x=importance['Gain'], y=importance['Feature'], orientation='h', marker_color='#3498db', name='Importancia'), row=2, col=1)

# D) Riesgo por Cluster
if not df_risk[df_risk['Es_Riesgo']==1].empty:
    riesgo_cluster = df_risk[df_risk['Es_Riesgo']==1]['Cluster'].value_counts().reset_index()
    riesgo_cluster.columns = ['Cluster', 'Clientes en Riesgo']
    fig.add_trace(go.Bar(x=riesgo_cluster['Cluster'], y=riesgo_cluster['Clientes en Riesgo'], marker_color='#e67e22', name='Riesgo'), row=2, col=2)
else:
    fig.add_annotation(text="Sin Riesgos Detectados", row=2, col=2, showarrow=False)

fig.update_layout(height=900, width=1200, title_text="<b>Tablero de Control de Retención (Modelo de Fuga)</b>", template="plotly_white")
fig.show()

Entrenando modelo definitivo...
¡Alerta! Se detectaron 150 clientes en riesgo de fuga.
Top 5 Clientes en Riesgo (Prioridad):
     ID_Cliente  Probabilidad_Fuga  Es_Riesgo Cluster  Monto_Total_Historico  \
5             6           0.963934          1       2                 184.61   
36           37           0.963872          1       2                 182.79   
76           77           0.961363          1       2                 190.19   
195         196           0.961212          1       2                  31.25   
88           89           0.961179          1       2                 218.14   

     Dias_Sin_Venir  
5                33  
36               37  
76               50  
195               2  
88               32  


In [31]:
import plotly.figure_factory as ff
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

# ==============================================================================
# VISUALIZACIÓN: MATRIZ DE CONFUSIÓN DE NEGOCIO
# ==============================================================================

print("Generando Matriz de Confusión (Validación Cruzada)...")

# 1. GENERAR PREDICCIONES HONESTAS (Cross-Validation)
# Usamos 'cross_val_predict' para simular cómo se comporta el modelo con datos que NO ha visto
# Esto es más realista que usar solo el set de prueba.
preds_cv_proba = cross_val_predict(
    model_final, 
    X, y, 
    cv=5, 
    method='predict_proba', 
    n_jobs=-1
)[:, 1]

# Aplicamos tu Umbral de Negocio (0.45)
preds_cv = (preds_cv_proba >= umbral_negocio).astype(int)

# 2. CALCULAR MATRIZ
cm = confusion_matrix(y, preds_cv)

# Convertir a porcentajes para que sea más fácil de leer
cm_text = [[str(y) for y in x] for x in cm] # Texto con números absolutos
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # Porcentajes por fila (Recall)

# 3. CREAR GRÁFICA (HEATMAP)
# Etiquetas de Negocio
x_labels = ['Pred: Se Queda', 'Pred: Fuga (Alerta)']
y_labels = ['Real: Se Queda', 'Real: Fuga']

# Configuración de colores y anotaciones
fig_cm = ff.create_annotated_heatmap(
    z=cm, 
    x=x_labels, 
    y=y_labels, 
    annotation_text=cm_text, 
    colorscale='Blues',
    showscale=True
)

# Personalizar diseño para impacto ejecutivo
fig_cm.update_layout(
    title_text=f"<b>Matriz de Confusión (Umbral: {umbral_negocio})</b><br>Evaluación de Precisión Operativa",
    height=500, width=600,
    xaxis=dict(title='Lo que dijo el Modelo', side='bottom'),
    yaxis=dict(title='La Realidad (Historia)'),
    template="plotly_white"
)

# Agregar anotaciones de interpretación (Insight directo en la gráfica)
fig_cm.add_annotation(
    x=1, y=1, 
    text=f"<b>RECUPERADOS<br>{cm_norm[1][1]:.1%}</b>", 
    showarrow=False, 
    font=dict(color="white", size=14),
    yshift=30
)
fig_cm.add_annotation(
    x=0, y=1, 
    text=f"Perdidos<br>{cm_norm[1][0]:.1%}", 
    showarrow=False, 
    font=dict(color="black", size=10),
    yshift=-30
)

fig_cm.show()

Generando Matriz de Confusión (Validación Cruzada)...
